In [1]:
# Sample program for one scrip
from ib_insync import *
util.startLoop()
# ib=IB().connect('127.0.0.1', 7496, clientId=9) # kavi tws live
ib = IB().connect('127.0.0.1', 4001, clientId=9) # kavi IBG live

In [10]:
%%time
import numpy as np
import pandas as pd
from itertools import product
import datetime

import utils # for catch function

# sd multiple for band
sigma = 2       # 2 sigma is about 95% probability
penalty = 1.8   # e.g. 1.2 is 20% above

# market
exchange = 'NSE'

#... prepare lot dataframe for underlying
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

# %%time
#... Get the scrip
symbol = 'RELIANCE'
contract = ib.qualifyContracts(Stock(symbol, exchange))[0]

#... Get volatility, hi52 and lo52
duration = '12 M'
size = '1 day'
bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr=duration, barSizeSetting=size, 
                     whatToShow='TRADES', useRTH=True, 
                     formatDate=1, keepUpToDate=True)

stDev = np.std(a=[b.close for b in bars], ddof=0)

hi52 = max([b.high for b in bars])
lo52 = min([b.low for b in bars])

meanPrice = np.mean([b.close for b in bars])

#... Get the lot and margin
lot = df_paisa.loc[df_paisa.ibSymbol == symbol, 'Mlot'].item()
margin = df_paisa.loc[df_paisa.ibSymbol == symbol, 'TotMgnPerLt'].item()

#... Get the option chain tickers
chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                      futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

expiries = set(*[c.expirations for c in chains])

cds = [ib.reqContractDetails(Option(symbol, e, exchange='NSE')) for e in expiries]

options = [c.contract for cs in cds for c in cs]

tickers = [t for i in range(0, len(options), 100) for t in ib.reqTickers(*options[i:i + 100])]    
ib.sleep(5)   # gives some time to fill the tickers    
tickers = [t for i in range(0, len(options), 100) for t in ib.reqTickers(*options[i:i + 100])]

# keep only those tickers with underlying prices
lib_t = {t: utils.catch(lambda: t.modelGreeks.undPrice) for t in tickers}
und_t = [k for k, v in lib_t.items() if v is not None]


#... Weed out unwanted SDs and make the option chain dataframe
# keep only the Ps and Cs outside the sigma band (95% probability)
tgt_ts = [t for t in und_t 
if ((t.contract.strike < t.modelGreeks.undPrice - stDev*sigma) & (t.contract.right == 'P')) | 
((t.contract.strike > t.modelGreeks.undPrice + stDev*sigma) & (t.contract.right == 'C'))]

# ... Build the dataframe
ts = [(t.contract.conId, t.contract.symbol, t.contract.lastTradeDateOrContractMonth, t.contract.strike, t.contract.right, t.modelGreeks.undPrice,
  t.contract.localSymbol, t.bid, t.bidSize, t.ask, t.askSize, t.close, t.modelGreeks.impliedVol, t.modelGreeks.delta, t.modelGreeks.optPrice, 
  t.modelGreeks.pvDividend, t.modelGreeks.gamma, t.modelGreeks.vega, t.modelGreeks.theta, lot, margin, hi52, lo52, meanPrice, stDev, t) for t in tgt_ts]

cols = ['conId', 'ibSymbol', 'expiry', 'strike', 'right', 'undPrice', 
        'localSymbol', 'bid', 'bidSize', 'ask', 'askSize', 'close', 'impliedVol', 'delta', 'optPrice', 
        'pvDividend', 'gamma', 'vega', 'theta', 'lot', 'margin', 'undHi', 'undLo', 'undMean', 'stdev', 'ticker']
df = pd.DataFrame(ts, columns=cols).sort_values(by=['expiry', 'strike'], ascending=False).reset_index(drop=True)

# Expected price is rounded up to 0.05 after putting the penalty
df['expPrice'] = round(df[['bid', 'ask', 'close']].max(axis=1)*penalty * 2, 1)/2

df['dte'] = (pd.to_datetime(df.expiry) - datetime.datetime.now()).dt.days

# make negative dtes to 1 to accommodate last day option expiries
df.loc[df.dte <= 0, 'dte'] = 1

df['rom'] = (df.expPrice*df.lot)/df.margin*252/df.dte

Wall time: 1min 35s


In [11]:
df.loc[:, ['ibSymbol', 'strike', 'expiry', 'dte', 'right', 'stDev', 'undLo', 'undHi', 'undPrice', 'undMean', 'bid', 'ask', 'close', 'expPrice', 'rom']]

,ibSymbol,strike,expiry,dte,right,stDev,undLo,undHi,undPrice,undMean,bid,ask,close,expPrice,rom
0,RELIANCE,1660.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.00,0.00,0.000000
1,RELIANCE,1640.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.00,0.00,0.000000
2,RELIANCE,1620.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.00,0.00,0.000000
3,RELIANCE,1600.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.00,0.00,0.000000
4,RELIANCE,1580.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.00,0.00,0.000000
5,RELIANCE,1560.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.00,0.00,0.000000
6,RELIANCE,1540.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.00,0.00,0.000000
7,RELIANCE,1520.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.05,0.10,0.130670
8,RELIANCE,1500.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.10,0.20,0.261340
9,RELIANCE,1480.0,20181227,1,C,NaN,871.45,1328.95,1089.15,1040.2025,-1.0,-1.0,0.25,0.45,0.588014
